## **K-NEAREST NEIGHBORS**
Từ một tập dữ liệu cho trước, khi có một dữ liệu mới, mô hình sẽ chọn ra K điểm dữ liệu gần nhất, sau đó xem xét nhãn của K điểm dữ liệu này và quyết định xem điểm mới này thuộc lớp nào?

#### Mô hình bài toán
Chúng ta có một các điểm:
$$
\mathbf{X} 
= 
\begin{bmatrix}
\mathbf{x}_1 \\
... \\
\mathbf{x}_N
\end{bmatrix}

\text{với} \space \mathbf{x}_i = [x_1, ..., x_d]
$$
Cùng với nhãn dữ liệu tương ứng:
$$
\mathbf{y} 
=
\begin{bmatrix}
y_1 \\
... \\
y_N
\end{bmatrix}
$$
Chúng ta có một điểm dữ liệu mới cần dự đoán:
$$
z = [z_1, ..., z_d]
$$

Ý tưởng của KNN.

Tính khoảng cách từ điểm $\mathbf{z}$ đến tất cả các điểm dữ liệu trong tập $\mathbf{X}$ 

$$
d_i = \|\mathbf{z} - \mathbf{x}_i\|_2^2
$$

$$
\mathbf{d} 
=
\begin{bmatrix}
\|\mathbf{z} - \mathbf{x}_1\|_2^2 \\
... \\
\|\mathbf{z} - \mathbf{x}_N\|_2^2
\end{bmatrix}
= 
\begin{bmatrix}
d_1 \\
... \\
d_N 
\end{bmatrix}
$$

Trong tập $\mathbf{d}$ chọn ra index của $K$ giá trị nhỏ nhất. $(*)$
$$
\mathbf{I} = [i_1, ..., i_k]
$$

Khi đó giá trị dự đoán của mô hình sẽ được quyết định bởi các phần tử trong tập:
$$
\mathbf{\bar{y}} = [y_{i_1}, ..., y_{i_k}]
$$

Giá trị dự đoán sẽ được tính như sau đối với bài toán Regresstion
$$
\hat{y} 
= \frac{w_1y_{i_1}+ ... + w_ky_{i_k}}{w_1 + ...+ w_k}
= \frac{\sum_{l=1}^k{w_ly_{i_l}}}{\sum_{l=1}^k{w_l}}
\text{, với }
w_i \text{ là trọng số}
$$

Đối với bài toán phân loại, giá trị dự đoán của mô hình sẽ là giá trị class xuất hiện nhiều nhất
$$
\hat{y} = \max{\{y_i : i = \bar{1..K}\}}
$$

Trong bước $(*)$ để tăng tốc độ tính toán, ta có thể giải một bài toán có cùng kết quả  
Xét
$$
d_i = \|\mathbf{z} - \mathbf{x}_i \|_2^2 
= (\mathbf{z} - \mathbf{x}_i)(\mathbf{z} - \mathbf{x}_i)^T 
= (\mathbf{z} - \mathbf{x}_i)(\mathbf{z}^T - \mathbf{x}_i^T)
= \mathbf{z}\mathbf{z}^T + \mathbf{x}_i\mathbf{x}_i^T - 2\mathbf{x}_i\mathbf{z}^T
= \|\mathbf{z}\|_2^2 + \|\mathbf{x}_i \|_2^2 - 2\mathbf{x}_i\mathbf{z}^T
$$

Gọi
$$
\mathbf{\bar{d}}
=
\begin{bmatrix}
\bar{d}_1 \\
... \\
\bar{d}_N
\end{bmatrix}
\text{với} \space
\bar{d}_i = \|\mathbf{x}_i \|_2^2 - 2\mathbf{x}_i\mathbf{z}^T
$$

Tiếp tục triển khai:
$$
\mathbf{\bar{d}}
=
\begin{bmatrix}
\|\mathbf{x}_1 \|_2^2 - 2\mathbf{x}_1\mathbf{z}^T \\
... \\
\|\mathbf{x}_N \|_2^2 - 2\mathbf{x}_N\mathbf{z}^T
\end{bmatrix}
=
\begin{bmatrix}
\|\mathbf{x}_1 \|_2^2\\
... \\
\|\mathbf{x}_N \|_2^2
\end{bmatrix}
-
2
\begin{bmatrix}
\mathbf{x}_1\mathbf{z}^T \\
... \\
\mathbf{x}_N\mathbf{z}^T
\end{bmatrix}
=
\begin{bmatrix}
\|\mathbf{x}_1 \|_2^2\\
... \\
\|\mathbf{x}_N \|_2^2
\end{bmatrix}
-
2\mathbf{X}\mathbf{z}
$$
Khi đó $(*)$ có cùng kết quả với bài toán Trong tập $\mathbf{\bar{d}}$ chọn ra index của $K$ giá trị nhỏ nhất.
Từ đó, ta có thể tính các giá trị $\|\mathbf{x}_i \|_2^2$ trước và có thể tính toán nhanh hơn

In [87]:
import numpy as np
from sklearn                 import neighbors, datasets
from sklearn.metrics         import accuracy_score
from sklearn.model_selection import train_test_split # for splitting data

In [88]:
class Model:
    def __init__(self, n_neighbors=1, p = 2, weights = 'uniform'):
        self.p = p
        self.X = None
        self.y = None
        self.w = weights
        self.k = n_neighbors
    
    def fit(self, data, target):
        self.X  = data
        self.y  = target
        self.X2 = np.sum(self.X * self.X, axis=1)

    def dist_ps_fast(self, z):
        if self.p == 1:
            return np.sum(np.abs(self.X - z), axis=1) 
        if self.p == 2:
            return self.X2 - 2 * self.X @ z
        
    def predict(self, dataset):
        result = []
        for z in dataset:
            dbar = self.dist_ps_fast(z)        
            I    = np.argsort(dbar)[:self.k]
            # set weights
            dis_k = np.array([dbar[idx] for idx in I])
            if self.w == 'uniform':
                weights = np.ones(self.k)
            elif self.w == 'distance':
                weights = 1/(dis_k + 1e-9)  
            elif callable(self.w):
                weights = self.w(dis_k) + 1e-9
                #print(weights)
            else:
                raise ValueError("Invalid weight function")
            ybar = np.array([self.y[idx] for idx in I])
            yhat = round(np.sum(ybar * weights) / np.sum(weights))
            result.append(yhat)
        return np.array(result)

In [89]:
np.random.seed(7)
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size=130)

#### Sử dụng Model triển khai bên trên

In [90]:
model = Model(n_neighbors=1, p=2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"Accuracy of 1NN: {100*accuracy_score(y_test, y_pred)}")

Accuracy of 1NN: 92.3076923076923


#### Sử dụng Scikit-Learn

In [91]:
model = neighbors.KNeighborsClassifier(n_neighbors=1, p=2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"Accuracy of 1NN: {100*accuracy_score(y_test, y_pred)}")

Accuracy of 1NN: 92.3076923076923


### Trọng số

Với thuật toán trước đó K hàng xóm gần điểm sẽ tiến hành biểu quyết, giá trị của các lá phiếu là công bằng như sau.  

Tuy nhiên, chúng ta cũng có thể nghĩ rằng, những hàng xóm gần điểm dữ liệu hơn sẽ đáng tin cậy hơn, và ta có thể cho rằng lá phiếu của những người gần nhất sẽ đáng tin cậy nhất

#### Trọng số dựa trên khoảng cách

#### Sử dụng Model triển khai

In [92]:
model = Model(n_neighbors = 7, p = 2, weights='distance')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"Accuracy of 7NN (1/distance weights): {100*accuracy_score(y_test, y_pred):.2f} %")

Accuracy of 7NN (1/distance weights): 93.85 %


#### Sử dụng Scikit-Learn

In [93]:
model = neighbors.KNeighborsClassifier(n_neighbors = 7, p = 2, weights='distance')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"Accuracy of 7NN (1/distance weights): {100*accuracy_score(y_test, y_pred):.2f} %")

Accuracy of 7NN (1/distance weights): 94.62 %


### Trọng số tự định nghĩa

Một cách đánh trọng số khác thường được sử dụng

$$
w_i = \exp\left( \frac{-\|\mathbf{z} - \mathbf{x}_i \|_2^2}{\sigma^2} \right)
$$

In [94]:
def myweight(distances, sigma = 0.7):
    return np.exp(-distances**2/sigma**2)

#### Sử dụng Model triển khai bên trên

In [95]:
model = Model(n_neighbors = 7, p = 2, weights = myweight)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"Accuracy of 7NN (customized weights): {100*accuracy_score(y_test, y_pred):.2f} %")

Accuracy of 7NN (customized weights): 93.85 %


#### Sử dụng Scikit-Learn

In [96]:

model = neighbors.KNeighborsClassifier(n_neighbors = 7, p = 2, weights = myweight)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"Accuracy of 7NN (customized weights): {100*accuracy_score(y_test, y_pred):.2f} %")

Accuracy of 7NN (customized weights): 94.62 %


## Thảo luận
- KNN không phù hợp với bộ dữ liệu lớn